In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import gridspec

from scipy.interpolate import InterpolatedUnivariateSpline

from matplotlib import figure

import ipywidgets as widgets
from ipywidgets import interact, interactive
from ipywidgets import HBox, VBox, Box
from ipywidgets import GridspecLayout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Text, ButtonStyle

from operator import itemgetter
from matplotlib.ticker import MaxNLocator

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))



In [2]:
SNeff_path = 'MDet_yrs_vs_SNeff.csv'
SNeff_file = open(SNeff_path, "r")

MDet_years1 = []
SNeffs1 = []

lines = SNeff_file.readlines()
for line in lines[6:36]:
    vals = line.split(',')
    MDet_years1.append(float(vals[2]))
    SNeffs1.append(float(vals[3]))
    

MDet_to_SNeff = InterpolatedUnivariateSpline(MDet_years1, SNeffs1)

In [3]:
Freeze_path = 'Freeze_outs.csv'
Freeze_file = open(Freeze_path, "r")

SNeffs2 = []
spin0 = []
spin05 = []
spin1 = []

lines = Freeze_file.readlines()
for line in lines[10:]:
    vals = line.split(',')
    SNeffs2.append(float(vals[0]))
    
    spin0.append(float(vals[1]))
    spin05.append(float(vals[2]) if len(vals[2]) > 0 else np.nan)
    spin1.append(float(vals[3]) if len(vals[3]) > 0 else np.nan)

SNeff_to_spin0 = InterpolatedUnivariateSpline(SNeffs2, spin0)
SNeff_to_spin05 = InterpolatedUnivariateSpline(SNeffs2[20:], spin05[20:])
SNeff_to_spin1 = InterpolatedUnivariateSpline(SNeffs2[27:], spin1[27:])

In [4]:
def MDetYrs_to_Cluster_01(yrs):
    slope = 0.55
    inter = 4.74
    return( pow(10, inter + slope*np.log10(yrs)) )
    
def MDetYrs_to_Cluster_05(yrs):
    slope = 0.57
    inter = 4.62
    return( pow(10, inter + slope*np.log10(yrs)) )

def MDetYrs_to_Cluster_10(yrs):
    slope = 0.69
    inter = 4.18
    return( pow(10, inter + slope*np.log10(yrs)) )

def MDetYrs_to_Cluster_15(yrs):
    slope = 0.73
    inter = 3.53
    return( pow(10, inter + slope*np.log10(yrs)) )

def MDetYrs_to_Cluster_20(yrs):
    slope = 0.88
    inter = 2.78
    return( pow(10, inter + slope*np.log10(yrs)) )

In [34]:
Inflation_path = 'Inflation_science.csv'
Inflation_file = open(Inflation_path, "r")

OLD_Inflation_path = 'Inflation_science_OLD.csv'
OLD_Inflation_file = open(OLD_Inflation_path, "r")


SP_inf_yrs = []
Ch_inf_yrs = []

SP_A4 = []
SP_A7 = []
Ch_A4 = []
Ch_A7 = []


lines = Inflation_file.readlines()
for line in lines[1:]:
    vals = line.split(',')
    
    SP_inf_yrs.append(float(vals[1])/1000000)
    
    SP_A4.append(float(vals[2]))
    SP_A7.append(float(vals[3]))
    
OLD_lines = OLD_Inflation_file.readlines()
for line in OLD_lines[2:]:
    vals = line.split(',')
    
    Ch_inf_yrs.append(float(vals[1]))
    
    Ch_A4.append(float(vals[3]))
    Ch_A7.append(float(vals[5]))

slopeA4 = (SP_A4[-1] - SP_A4[-10])/(SP_inf_yrs[-1] - SP_inf_yrs[-10])
slopeA7 = (SP_A7[-1] - SP_A7[-10])/(SP_inf_yrs[-1] - SP_inf_yrs[-10])

#plt.plot(new_yrs, new_A4 )

for i in range(1,50):
    SP_inf_yrs.append(SP_inf_yrs[-1] + i*0.1)
    SP_A4.append(SP_A4[-1] + i*0.1*slopeA4)
    SP_A7.append(SP_A7[-1] + i*0.1*slopeA7)

MDetYrs_to_SP_A4 = InterpolatedUnivariateSpline(SP_inf_yrs, SP_A4)
MDetYrs_to_SP_A7 = InterpolatedUnivariateSpline(SP_inf_yrs, SP_A7)

MDetYrs_to_Ch_A4 = InterpolatedUnivariateSpline(Ch_inf_yrs, Ch_A4)
MDetYrs_to_Ch_A7 = InterpolatedUnivariateSpline(Ch_inf_yrs, Ch_A7)

In [35]:
def make_plot(SO_eff,
              
              SO_Used, 
              SO_Start,
              
              AdvSO_Used, 
              AdvSO_Start, 
              AdvSO_End,
              
              CHLAT1_eff,
              
              CHLAT1_Used, 
              CHLAT1_Start, 
              CHLAT1_End,
              
              CHALT2_Used, 
              CHLAT2_Start, 
              CHLAT2_End,
                 
              y_axis,
              
              z01,
              z05,
              z10,
              z15,
              z20,
              
              South_Used, 
              South_Start,
              South_End,
              South_con,

              Bicep_Used, 
              Bicep_End,
              Bicep_eff,


              Chile_Used,   
              Chile_Start,
              Chile_End,
              Chile_con,

              SO_Used_2, 
              SO_End_2,
              SO_eff_2,
              
              x_axis,
              
              Save,
              Update):
    
    
    
    if AdvSO_Used:
        SO_Duration = AdvSO_Start - SO_Start
    
    Detectors    = [0.031, 0.062  , 0.135    , 0.135    ] 
    Efficiencies = [SO_eff , SO_eff , CHLAT1_eff , CHLAT1_eff ]
    Names        = ["SO" , "AdvSO", "CHLAT 1", "CHLAT 2"]
    
    SO_End = AdvSO_Start
    if AdvSO_Used == False:
        SO_End = AdvSO_End
        
    StartYears   = [SO_Start, AdvSO_Start,  CHLAT1_Start,  CHLAT2_Start]
    EndYears     = [SO_End,  AdvSO_End,  CHLAT1_End,  CHLAT2_End]
    Used         = [SO_Used,  AdvSO_Used,  CHLAT1_Used,  CHALT2_Used]
    
    increment = 1/52
        
    if (Used[0] or Used[1] or Used[2] or Used[3]):
        used_map = map(int, Used)
        used_int = list(used_map)
        st_years = np.multiply(StartYears, used_int)
        st_years = st_years[st_years != 0]
        FirstYear = np.min(st_years)
        LastYear = np.max(np.multiply(EndYears, used_int))

       

        years = np.arange(FirstYear, LastYear, increment)
        det_times = []
        total_det_time = 0

        for date in years:
            for i in range(len(Detectors)):
                if Used[i]:
                    if date >= StartYears[i] and date <= EndYears[i]:
                        total_det_time += Detectors[i]*Efficiencies[i]*increment

            det_times.append(total_det_time)

        These_SNeffs = MDet_to_SNeff(det_times)

        These_spin0s = SNeff_to_spin0(These_SNeffs)
        These_spin05s = SNeff_to_spin05(These_SNeffs)
        These_spin1s = SNeff_to_spin1(These_SNeffs)
        
        These_z01s = MDetYrs_to_Cluster_01(det_times)
        These_z05s = MDetYrs_to_Cluster_05(det_times)
        These_z10s = MDetYrs_to_Cluster_10(det_times)
        These_z15s = MDetYrs_to_Cluster_15(det_times)
        These_z20s = MDetYrs_to_Cluster_20(det_times)
        

    #===============================================================================================
    
    StartYears_2   = [2024, 2029,  2029,  2029]
    Durations_2    = [5,  10,  7,  7]
    Used_2         = [False,  False,  True,  True]
    Efficiencies_2 = [0.9, 0.9, 1.0, 1.0]

    used_map_2 = map(int, Used_2)
    used_int_2 = list(used_map_2)
    st_years_2 = np.multiply(StartYears_2, used_int_2)
    st_years_2 = st_years_2[st_years_2 != 0]
    FirstYear_2 = np.min(st_years_2)
    LastYear_2 = np.max(np.multiply(np.add(StartYears_2,Durations_2), used_int_2))

    years_2 = np.arange(FirstYear_2, LastYear_2, increment)
    det_times_2 = []
    total_det_time_2 = 0

    for date_2 in years_2:
        for i in range(len(Detectors)):
            if Used_2[i]:
                if date_2 >= StartYears_2[i] and date_2 <= StartYears_2[i] + Durations_2[i]:
                    total_det_time_2 += Detectors[i]*Efficiencies_2[i]*increment

        det_times_2.append(total_det_time_2)

    These_SNeffs_2 = MDet_to_SNeff(det_times_2)

    These_spin0s_2 = SNeff_to_spin0(These_SNeffs_2)
    These_spin05s_2 = SNeff_to_spin05(These_SNeffs_2)
    These_spin1s_2 = SNeff_to_spin1(These_SNeffs_2)
    
    These_z01s_2 = MDetYrs_to_Cluster_01(det_times_2)
    These_z05s_2 = MDetYrs_to_Cluster_05(det_times_2)
    These_z10s_2 = MDetYrs_to_Cluster_10(det_times_2)
    These_z15s_2 = MDetYrs_to_Cluster_15(det_times_2)
    These_z20s_2 = MDetYrs_to_Cluster_20(det_times_2)

    
    #===============================================================================================
    
    South_dets   = 0.281
    Bicep_dets   = 0.051
    
    increment = 1/52
    
    inf_PDB_years = np.arange(2029, 2036, increment)
    inf_PDB_det_times = []
    inf_PDB_total_det_time = 0
    for date in inf_PDB_years:
        if date > 2029 and date < 2036:
            inf_PDB_total_det_time += South_dets*1.0*increment


        inf_PDB_det_times.append(inf_PDB_total_det_time)

    inf_PDB_A4 = MDetYrs_to_SP_A4(inf_PDB_det_times)
    inf_PDB_A7 = MDetYrs_to_SP_A7(inf_PDB_det_times)
        
    
    
    start_year = South_Start
    if Bicep_Used:
        start_year = 2020
        
    
    end_year = max(South_End , Bicep_End)
    if not South_Used: 
        end_year = Bicep_End
    if not Bicep_Used:
        end_year = South_End 
    
    
    if South_Used or Bicep_Used: 
        south_years = np.arange(start_year, end_year, increment)
        south_det_times = []
        south_total_det_time = 0

        for date in south_years:
            if South_Used and date > South_Start and date < South_End:
                south_total_det_time += South_dets*South_con*increment

            if Bicep_Used and date > 2020 and date < Bicep_End:
                south_total_det_time += Bicep_dets*Bicep_eff*increment

            south_det_times.append(south_total_det_time)
       
        
        This_SP_A4 = MDetYrs_to_SP_A4(south_det_times)
        This_SP_A7 = MDetYrs_to_SP_A7(south_det_times)
      

    # = = = = = = = = = = = = = = = = = = 
    
    Chile_dets   = 0.15
    SO_dets      = 0.01
    
    increment = 1/52
    
    if SO_Used_2:
        start_year = 2024
    else:
        start_year = Chile_Start
        
    if SO_Used_2 and Chile_Used:
        end_year = max(Chile_End, SO_End)
    elif Chile_Used:
        end_year = Chile_End
    elif SO_Used_2:
        end_year = SO_End
    
    if Chile_Used or SO_Used_2: 
        chile_years = np.arange(start_year, end_year, increment)

        chile_det_times = []
        chile_total_det_time = 0

        for date in chile_years:
            if Chile_Used and date > Chile_Start and date < Chile_End:
                chile_total_det_time += Chile_dets*Chile_con*increment

            if SO_Used_2 and date > 2024 and date < SO_End:
                chile_total_det_time += SO_dets*SO_eff*increment        

            chile_det_times.append(chile_total_det_time)

        This_Ch_A4 = MDetYrs_to_Ch_A4(chile_det_times)
        This_Ch_A7 = MDetYrs_to_Ch_A7(chile_det_times)
    
    #===============================================================================================


    %matplotlib inline

    make_plot.fig = plt.figure(figsize=(15, 10), dpi=80, constrained_layout=False)
    #fig.suptitle('Time-series Comparason', fontsize=24)
    gs1 = make_plot.fig.add_gridspec(nrows = 1, ncols = 1, wspace=0.001) 

    ax0 = plt.subplot(gs1[0, 0])
    
    ax0.set_xlabel('Year', fontsize = 30)
   
    ax0.tick_params(axis='x', labelsize=23)
    ax0.tick_params(axis='y', labelsize=23)
    
    labels = False
    
    if(x_axis == 'Year'):
        
        if(y_axis == 'Light Relics: Freeze-Out Temperature'):

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                ax0.plot(years[:], These_spin1s[:], color='darkviolet', linestyle='dotted')
                ax0.plot(years[:], These_spin05s[:], color='blue', linestyle='dotted')
                ax0.plot(years[:], These_spin0s[:], color='green', linestyle='dotted')

            ax0.plot(years_2[:], These_spin1s_2[:], color='darkviolet', linestyle='solid')
            ax0.plot(years_2[:], These_spin05s_2[:], color='blue', linestyle='solid')
            ax0.plot(years_2[:], These_spin0s_2[:], color='green', linestyle='solid')

            ax0.plot([2022,2023,2024], [1,2,3], c = "black", linestyle='solid', label='CMB-S4 PBD')
            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1: ax0.plot([2022,2023,2024], [1,2,3], c = "black", linestyle='dotted', label='CUSTOM')
            ax0.plot([2022,2023,2024], [1,2,3], color='darkviolet', linestyle='solid', label='Spin 1')
            ax0.plot([2022,2023,2024], [1,2,3], color='blue', linestyle='solid', label='Spin 1/2')
            ax0.plot([2022,2023,2024], [1,2,3], color='green', linestyle='solid', label='Spin 0')


            anot = 300 #max(These_spin1s_2[-1], These_spin1s[-1])
            ax0.plot((2010,2050), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='QCD Phase Transition')
            #ax0.text(2041.9, anot, "QCD Phase Transition", c='#bfbfbf', fontsize=15, transform=ax0.transData)

            ax0.set_ylabel('Freeze Out Temp [MeV]', fontsize = 30)
            ax0.set_ylim(80,300000)
            ax0.set_yscale('log')
            ax0.legend()

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                year_start = min(years[0], years_2[0]) - 0.5
                year_end = max(years[-1], years_2[-1]) + 0.5
            else:
                year_start = years_2[0] - 0.5
                year_end = years_2[-1] + 0.5
            ax0.set_xlim(year_start, year_end)



        elif(y_axis == 'Light Relics: σ(N_eff)'):

            ax0.plot(years_2[:], These_SNeffs_2[:], label='CMB-S4 PBD', color='blue', linestyle='solid')

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                ax0.plot(years[:], These_SNeffs[:], label='Custom', color='blue', linestyle='dotted')

            anot = 0.030
            ax0.plot((2010, 2050), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='CMB-S4 Target')

            ax0.set_ylabel('σ(N_eff)', fontsize = 30)
            ax0.set_ylim(0.025,0.05)
            ax0.legend()

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                year_start = min(years[0], years_2[0]) - 0.5
                year_end = max(years[-1], years_2[-1]) + 0.5
            else:
                year_start = years_2[0] - 0.5
                year_end = years_2[-1] + 0.5
            ax0.set_xlim(year_start, year_end)



        elif(y_axis == 'Clusters: Chile Counts'):
            ax0.plot([2022,2023,2024], [1,2,3], c = "black", linestyle='solid', label='CMB-S4 PBD')
            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1: ax0.plot([2022,2023,2024], [1,2,3], c = "black", linestyle='dotted', label='Custom')


            if z01: ax0.plot([2022,2023,2024], [1,2,3], color='darkviolet', linestyle='solid', label='z≥0.1')
            if z05: ax0.plot([2022,2023,2024], [1,2,3], color='blue', linestyle='solid', label='z≥0.5')
            if z10: ax0.plot([2022,2023,2024], [1,2,3], color='green', linestyle='solid', label='z≥1.0')
            if z15: ax0.plot([2022,2023,2024], [1,2,3], color='orange', linestyle='solid', label='z≥1.5')
            if z20: ax0.plot([2022,2023,2024], [1,2,3], color='red', linestyle='solid', label='z≥2.0')

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                if z01: ax0.plot(years[:], These_z01s[:], color='darkviolet', linestyle='dotted')
                if z05: ax0.plot(years[:], These_z05s[:], color='blue', linestyle='dotted')
                if z10: ax0.plot(years[:], These_z10s[:], color='green', linestyle='dotted')
                if z15: ax0.plot(years[:], These_z15s[:], color='orange', linestyle='dotted')
                if z20: ax0.plot(years[:], These_z20s[:], color='red', linestyle='dotted')

            if z01: ax0.plot(years_2[:], These_z01s_2[:], color='darkviolet', linestyle='solid')
            if z05: ax0.plot(years_2[:], These_z05s_2[:], color='blue', linestyle='solid')
            if z10: ax0.plot(years_2[:], These_z10s_2[:], color='green', linestyle='solid')
            if z15: ax0.plot(years_2[:], These_z15s_2[:], color='orange', linestyle='solid')
            if z20: ax0.plot(years_2[:], These_z20s_2[:], color='red', linestyle='solid')

            ax0.set_ylabel('Number of Clusters', fontsize = 30)
            ax0.set_ylim(10,200000)
            ax0.set_yscale('log')
            ax0.legend()

            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                year_start = min(years[0], years_2[0]) - 0.5
                year_end = max(years[-1], years_2[-1]) + 0.5
            else:
                year_start = years_2[0] - 0.5
                year_end = years_2[-1] + 0.5
            ax0.set_xlim(year_start, year_end)



        elif(y_axis == 'Inflation: σ(r=0)' and True):

            ax0.fill_between(inf_PDB_years[:], inf_PDB_A4[:], inf_PDB_A7[:], color='blue', alpha=0.65, label='PDB')
            ax0.plot(inf_PDB_years[:], inf_PDB_A4[:], color='blue', linestyle='solid')
            ax0.plot(inf_PDB_years[:], inf_PDB_A7[:], color='blue', linestyle='solid')

            if South_Used or Bicep_Used:
                ax0.fill_between(south_years[:], This_SP_A4[:], This_SP_A7[:], color='blue', alpha=0.3, label='Custom')
                ax0.plot(south_years[:], This_SP_A4[:], color='blue', linestyle='dotted')
                ax0.plot(south_years[:], This_SP_A7[:], color='blue', linestyle='dotted')

            #if South_Used or Bicep_Used: 
                #ax0.plot([2022,2023,2024], [-1,-2,-3], color="#bfbfbf", linestyle='solid', label='South Pole')
                #if A4: ax0.plot(south_years[:], This_SP_A4[:], color='blue', linestyle='dotted')
                #if A7: ax0.plot(south_years[:], This_SP_A7[:], color='green', linestyle='dotted')

            #if Chile_Used or SO_Used_2: 
                #ax0.plot([2022,2023,2024], [-1,-2,-3], color="#bfbfbf", linestyle='dotted', label='Chile')
                #if A4: ax0.plot(chile_years[:], This_Ch_A4[:], color='blue', linestyle='solid')
                #if A7: ax0.plot(chile_years[:], This_Ch_A7[:], color='green', linestyle='solid')

            #if A4: ax0.plot([2022,2023,2024], [-1,-2,-3], color='blue', linestyle='solid', label='A4')
            #if A7: ax0.plot([2022,2023,2024], [-1,-2,-3], color='green', linestyle='solid', label='A7')

            #ax0.plot((2020, 2041.8), (5, 5), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)) )
            #ax0.text(2041.9, 5, "CMB-S4 Target", c='#bfbfbf', fontsize=15, transform=ax0.transData)

            anot = 5e-4
            ax0.plot((2010, 2050), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='CMB-S4 Target')

            ax0.set_ylabel('σ(r=0)', fontsize = 30)
            ax0.set_yscale('log')
            ax0.set_ylim(0.0002,0.05)
            ax0.set_xlabel('Year', fontsize = 30)
            ax0.legend()
            ax0.tick_params(axis='x', labelsize=23)
            ax0.tick_params(axis='y', labelsize=23)

            if South_Used or Bicep_Used:
                year_start = min(inf_PDB_years[0], south_years[0]) - 0.5
                year_end = max(inf_PDB_years[-1], south_years[-1], Bicep_End) + 0.5
            else:
                year_start = inf_PDB_years[0] - 0.5
                year_end = inf_PDB_years[-1] + 0.5
            ax0.set_xlim(year_start, year_end)
            #ax0.set_xticklabels(np.arange(int(year_start+0.5), int(year_end-0.5)))


        else:
            ax0.text(2027, 0.5, "In Preparation", c='#bfbfbf', fontsize=40, transform=ax0.transData)
            ax0.set_xlim(2020, 2040)
            ax0.set_ylim(0, 1)
            ax0.axis('off')
    
    if(x_axis == 'Detector Years'):
        
        if(y_axis == 'Light Relics: Freeze-Out Temperature'):
            
            Relics_det_yrs = np.arange(0,det_times_2[-1], 0.02)
            MDet_SNeffs = MDet_to_SNeff(Relics_det_yrs)
            MDet_spin0s = SNeff_to_spin0(MDet_SNeffs)
            MDet_spin05s = SNeff_to_spin05(MDet_SNeffs)
            MDet_spin1s = SNeff_to_spin1(MDet_SNeffs)
            ax0.plot(Relics_det_yrs[:], MDet_spin1s[:], color='darkviolet', linestyle='solid', label='Spin 1')
            ax0.plot(Relics_det_yrs[:], MDet_spin05s[:], color='blue', linestyle='solid', label='Spin 1/2')
            ax0.plot(Relics_det_yrs[:], MDet_spin0s[:], color='green', linestyle='solid', label='Spin 0')
            
            ax0.plot( (0, 0.1), (0, 0.1), color='black', linestyle='solid', label='PDB')
            
            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                Relics_det_yrs = np.arange(0,det_times[-1], 0.02)
                MDet_SNeffs = MDet_to_SNeff(Relics_det_yrs)
                MDet_spin0s = SNeff_to_spin0(MDet_SNeffs)
                MDet_spin05s = SNeff_to_spin05(MDet_SNeffs)
                MDet_spin1s = SNeff_to_spin1(MDet_SNeffs)
                ax0.plot(Relics_det_yrs[:], MDet_spin1s[:], color='darkviolet', linestyle='dotted')
                ax0.plot(Relics_det_yrs[:], MDet_spin05s[:], color='blue', linestyle='dotted')
                ax0.plot(Relics_det_yrs[:], MDet_spin0s[:], color='green', linestyle='dotted')
                
                ax0.plot( (0, 0.1), (0, 0.1), color='black', linestyle='dotted', label='Custom')
            
            anot = 300 #max(These_spin1s_2[-1], These_spin1s[-1])
            ax0.plot((0,3), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='QCD Phase Transition')
            #ax0.text(2041.9, anot, "QCD Phase Transition", c='#bfbfbf', fontsize=15, transform=ax0.transData)

            ax0.set_ylabel('Freeze Out Temp [MeV]', fontsize = 30)
            ax0.set_ylim(80,300000)
            ax0.set_yscale('log')
            ax0.set_xlabel('Millions of Detector Years', fontsize = 30)
            ax0.legend()
            

        elif(y_axis == 'Light Relics: σ(N_eff)'):
            
            Relics_det_yrs = np.arange(0,det_times_2[-1], 0.02)
            MDet_SNeffs = MDet_to_SNeff(Relics_det_yrs)
            ax0.plot(Relics_det_yrs[:], MDet_SNeffs[:], color='blue', linestyle='solid', label='PDB')
            
            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                Relics_det_yrs = np.arange(0,det_times[-1], 0.02)
                MDet_SNeffs = MDet_to_SNeff(Relics_det_yrs)
                ax0.plot(Relics_det_yrs[:], MDet_SNeffs[:], color='blue', linestyle='dotted', label='Custom')

                
            anot = 0.030
            ax0.plot((0,3), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='CMB-S4 Target')

            ax0.set_ylabel('σ(N_eff)', fontsize = 30)
            ax0.set_ylim(0.025,0.050)
            ax0.set_xlabel('Millions of Detector Years', fontsize = 30)
            ax0.legend()

            
        elif(y_axis == 'Clusters: Chile Counts'):
            
            Cluster_det_yrs = np.arange(0.01 ,det_times_2[-1], 0.005)
            MDet_z01s_2 = MDetYrs_to_Cluster_01(Cluster_det_yrs)
            MDet_z05s_2 = MDetYrs_to_Cluster_05(Cluster_det_yrs)
            MDet_z10s_2 = MDetYrs_to_Cluster_10(Cluster_det_yrs)
            MDet_z15s_2 = MDetYrs_to_Cluster_15(Cluster_det_yrs)
            MDet_z20s_2 = MDetYrs_to_Cluster_20(Cluster_det_yrs)
        

            if z01: ax0.plot(Cluster_det_yrs[:], MDet_z01s_2[:], color='darkviolet', linestyle='solid', label='z≥0.1')
            if z05: ax0.plot(Cluster_det_yrs[:], MDet_z05s_2[:], color='blue', linestyle='solid', label='z≥0.5')
            if z10: ax0.plot(Cluster_det_yrs[:], MDet_z10s_2[:], color='green', linestyle='solid', label='z≥1.0')
            if z15: ax0.plot(Cluster_det_yrs[:], MDet_z15s_2[:], color='orange', linestyle='solid', label='z≥1.5')
            if z20: ax0.plot(Cluster_det_yrs[:], MDet_z20s_2[:], color='red', linestyle='solid', label='z≥2.0')
            
            ax0.plot( (0, 0.1), (0, 0.1), color='black', linestyle='solid', label='PDB')
            
            if (Used[0] or Used[1] or Used[2] or Used[3]) and len(det_times) > 1:
                Cluster_det_yrs = np.arange(0.01 ,det_times[-1], 0.005)
                MDet_z01s_2 = MDetYrs_to_Cluster_01(Cluster_det_yrs)
                MDet_z05s_2 = MDetYrs_to_Cluster_05(Cluster_det_yrs)
                MDet_z10s_2 = MDetYrs_to_Cluster_10(Cluster_det_yrs)
                MDet_z15s_2 = MDetYrs_to_Cluster_15(Cluster_det_yrs)
                MDet_z20s_2 = MDetYrs_to_Cluster_20(Cluster_det_yrs)


                if z01: ax0.plot(Cluster_det_yrs[:], MDet_z01s_2[:], color='darkviolet', linestyle='dotted')
                if z05: ax0.plot(Cluster_det_yrs[:], MDet_z05s_2[:], color='blue', linestyle='dotted')
                if z10: ax0.plot(Cluster_det_yrs[:], MDet_z10s_2[:], color='green', linestyle='dotted')
                if z15: ax0.plot(Cluster_det_yrs[:], MDet_z15s_2[:], color='orange', linestyle='dotted')
                if z20: ax0.plot(Cluster_det_yrs[:], MDet_z20s_2[:], color='red', linestyle='dotted')
                
                ax0.plot( (0, 0.1), (0, 0.1), color='black', linestyle='dotted', label='Custom')
            
            ax0.set_ylabel('Number of Clusters', fontsize = 30)
            ax0.set_ylim(10,200000)
            ax0.set_yscale('log')
            ax0.set_xlabel('Millions of Detector Years', fontsize = 30)
            ax0.legend()

        
        elif(y_axis == 'Inflation: σ(r=0)' and True):
            
            Inflation_det_yrs = np.arange(0,inf_PDB_det_times[-1], 0.01)
            MDet_A4 = MDetYrs_to_SP_A4(Inflation_det_yrs)
            MDet_A7 = MDetYrs_to_SP_A7(Inflation_det_yrs)

            ax0.fill_between(Inflation_det_yrs[:], MDet_A4[:], MDet_A7[:], color='blue', alpha=0.65, label='PDB')
            ax0.plot(Inflation_det_yrs[:], MDet_A4[:], color='blue', linestyle='solid')
            ax0.plot(Inflation_det_yrs[:], MDet_A7[:], color='blue', linestyle='solid')
            
            if (South_Used or Bicep_Used) and len(south_det_times) > 2:
                Inflation_det_yrs = np.arange(0, south_det_times[-1], 0.01)
                MDet_A4 = MDetYrs_to_SP_A4(Inflation_det_yrs)
                MDet_A7 = MDetYrs_to_SP_A7(Inflation_det_yrs)
                ax0.fill_between(Inflation_det_yrs[:], MDet_A4[:], MDet_A7[:], color='blue', alpha=0.3, label='Custom')
                ax0.plot(Inflation_det_yrs[:], MDet_A4[:], color='blue', linestyle='dotted')
                ax0.plot(Inflation_det_yrs[:], MDet_A7[:], color='blue', linestyle='dotted')

            anot = 5e-4
            ax0.plot((0, 3), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)), label='CMB-S4 Target')

            ax0.set_ylabel('σ(r=0)', fontsize = 30)
            ax0.set_yscale('log')
            ax0.set_ylim(0.0002,0.05)
            ax0.set_xlabel('Millions of Detector Years', fontsize = 30)
            ax0.legend()
            ax0.tick_params(axis='x', labelsize=23)
            ax0.tick_params(axis='y', labelsize=23)
            
            
        else:
            ax0.text(2027, 0.5, "In Preparation", c='#bfbfbf', fontsize=40, transform=ax0.transData)
            ax0.set_xlim(2020, 2040)
            ax0.set_ylim(0, 1)
            ax0.axis('off')
    
    # fig.tight_layout()
    #fig.savefig("Test-Timeline-Graph.png")
    if(x_axis == 'Year'): ax0.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.show()
    return make_plot.fig
    
    
    
    
w = interactive(make_plot, 
              
              SO_eff            = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),

              SO_Used           = widgets.Checkbox(value=False, description='Baseline', indent=False), 
              SO_Start          = widgets.Dropdown(options = np.arange(2024,2030,1), value=2024, description='Start Year:'),
                
              AdvSO_Used        = widgets.Checkbox(value=False, description='Advanced', indent=False) , 
              AdvSO_Start       = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              AdvSO_End         = widgets.Dropdown(options = np.arange(2029,2046,1), value=2029, description='End Year:'),
              
              CHLAT1_eff        = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),
             
              CHLAT1_Used       = widgets.Checkbox(value=False, description='CHLAT #1', indent=False) , 
              CHLAT1_Start      = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              CHLAT1_End        = widgets.Dropdown(options = np.arange(2030,2046,1), value=2036, description='End Year:'),

              CHALT2_Used       = widgets.Checkbox(value=False, description='CHLAT #2', indent=False), 
              CHLAT2_Start      = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'), 
              CHLAT2_End        = widgets.Dropdown(options = np.arange(2030,2046,1), value=2036, description='End Year:'),

              y_axis            = widgets.Dropdown(options=['Inflation: σ(r=0)', 'Light Relics: σ(N_eff)', 'Light Relics: Freeze-Out Temperature', 'Clusters: Chile Counts', 'Clusters: South Pole Counts'], value='Light Relics: Freeze-Out Temperature', description='Y Axis:', disabled=False), 

              z01               = widgets.Checkbox(value=True, description='z≥0.1', indent=False, width='3ex') , 
              z05               = widgets.Checkbox(value=True, description='z≥0.5', indent=False) , 
              z10               = widgets.Checkbox(value=True, description='z≥1.0', indent=False) , 
              z15               = widgets.Checkbox(value=True, description='z≥1.5', indent=False) , 
              z20               = widgets.Checkbox(value=True, description='z≥2.0', indent=False) , 
                
              
                
              South_Used        = widgets.Checkbox(value=False, description='CMB-S4', indent=False), 
              South_Start       = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              South_End         = widgets.Dropdown(options = np.arange(2030,2041,1), value=2036, description='End Year:'),
              South_con         = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),
                
              Bicep_Used        = widgets.Checkbox(value=False, description='South Pole Observatory', indent=False), 
              Bicep_End         = widgets.Dropdown(options = np.arange(2024,2041,1), value=2024, description='End Year:'),
              Bicep_eff         = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),
              
                
              Chile_Used        = widgets.Checkbox(value=False, description='CMB-S4', indent=False),   
              Chile_Start       = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              Chile_End         = widgets.Dropdown(options = np.arange(2030,2046,1), value=2036, description='End Year:'),
              Chile_con         = widgets.FloatSlider(value=1.0, min=0.5, max=1.0, step=0.05, description=' ' , indent=False, readout_format=".0%"),
                
              SO_Used_2         = widgets.Checkbox(value=False, description='Simons Observatory', indent=False), 
              SO_End_2          = widgets.Dropdown(options = np.arange(2024,2046,1), value=2029, description='End Year:'),
              SO_eff_2          = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),
                
              x_axis            = widgets.Dropdown(options=['Year', 'Detector Years'], value='Year', description='X Axis:', disabled=False), 
                
              Save              = widgets.ToggleButton(value=False, description='Export'),
              Update            = widgets.ToggleButton(value=False, description='Run'))


w.children

# This doesn't display the plot until one of the widgets has been used
#display(HBox([VBox(list(itemgetter(0,-1,-2)(w.children))), VBox(w.children[1:-2])]))
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='3px solid',
                    width='100%',
                    visibility = 'visible')

top_bar_visible = Layout(visibility = 'visible',
                    display='flex',
                    flex_flow='row',
                    align_items='flex-start',
                    width='500px')

layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')
layout_none    = widgets.Layout(display = "none")
layout_block   = widgets.Layout(display = "block")


spacer = Layout(height='30px')
space = Layout(width='30px')
s1 = Box([], layout=spacer)
s2 = Box([], layout=spacer)
sp = Box([], layout=space)


sim_text = "Simons Observatory:"
sim_lab = widgets.HTML(value = f"<b><font size='4px'>{sim_text}</b>")
sim_box = VBox(w.children[1:3])
adv_sim_box = VBox(w.children[3:6])
eff_text = "Relative Efficiency:"
eff_lab = widgets.HTML(value = f"<b><font size='2px'>{eff_text}</b>")                      
SO_eff = w.children[0]                      
SO_box = VBox([sim_lab, sim_box, adv_sim_box, eff_lab, SO_eff], layout=box_layout)



CMB_text = "CMB-S4:"
chlat_1_lab = widgets.HTML(value = f"<b><font size='4px'>{CMB_text}</b>")
chlat_1_box = VBox(w.children[7:10])
b2 = VBox([chlat_1_lab, chlat_1_box])
chlat_2_box = VBox(w.children[10:13])
con_text = "Construction Fraction:"
con_lab = widgets.HTML(value = f"<b><font size='2px'>{con_text}</b>")                       
con = w.children[6]
CMB_box = VBox([b2, s2, chlat_2_box, eff_lab, con], layout=box_layout)



south_text = "South Pole:"
south_lab = widgets.HTML(value = f"<b><font size='4px'>{south_text}</b>")
south_box = VBox(w.children[19:22])
con_lab = widgets.HTML(value = f"<b><font size='2px'>{con_text}</b>")                       
south_con = w.children[22]
bicep_box = VBox(w.children[23:25])
eff_lab = widgets.HTML(value = f"<b><font size='2px'>{eff_text}</b>")
bicep_eff = w.children[25]                       
South_box = VBox([south_lab, south_box, eff_lab, south_con, bicep_box, eff_lab, bicep_eff], layout=box_layout)


chile_text = "Chile:"
chile_lab = widgets.HTML(value = f"<b><font size='4px'>{chile_text}</b>")
chile_box = VBox(w.children[26:29])                       
chile_con = w.children[29]
South_SO_box = VBox(w.children[30:32])
SO_eff = w.children[32]
Chile_box = VBox([chile_lab, chile_box, con_lab, chile_con, South_SO_box, eff_lab, SO_eff], layout=layout_hidden)


sidebar1 = VBox([SO_box, s1, CMB_box])
sidebar2 = VBox([South_box, s1, Chile_box], layout=layout_none)

col_1 = VBox([sidebar1, sidebar2])


y_ax = w.children[13]
x_ax = w.children[-4]

z_bar = HBox(list(itemgetter( 14, 15, 16, 17, 18)(w.children)),  layout=layout_none)
indent = Layout(width='42px')
ind = Box([], layout=indent)
redshift_bar = HBox([ind, z_bar])

#top_bar = HBox([y_ax, x_ax, sp, sp, z_bar])
top_bar = VBox([HBox([y_ax, x_ax]), redshift_bar])

#graph = VBox(list(itemgetter(-1, -2, -3)(w.children)))
graph = VBox(list(itemgetter(-1, -2)(w.children)))

download_text = "To download this graph as a .png file, right click on it and select \"Save image as...\" "
download_t = widgets.HTML(value = f"<b><font size='2px'>{download_text}</b>")

col_2 = VBox([top_bar, graph, download_t])

window = HBox([col_1, sp, col_2])

Title_text = "CMB-S4 Science Timeline:"
title = widgets.HTML(value = f"<b><font size='6px'>{Title_text}</b>")

sub_text = "This tool shows how CMB-S4 reaches its various science goals as a function of observing time, and allows the user to compare the Preliminary Baseline Design (PBD) with custom configurations that incorporate other hardware and/or change the scope of CMB-S4. Selecting the Y Axis sets the science case to be plotted; selecting the X Axis plots the data in Calendar or Detector Years; and selecting a combination of telescopes, with their periods of operation and their per-detector efficiencies relative to the PBD, defines the custom configuration."
subtitle = widgets.HTML(value = f"<font size='2px'>{sub_text}")

ack_text = "This work relied on forecasts produced by the CMB-S4 Analysis Working Groups. Forecasts for other experiments are simply scaled by detector-count from CMB-S4, and may therefore disagree with the experiment’s own more detailed forecasts."
ack = widgets.HTML(value = f"<font size='2px'>{ack_text}")

con_text = "Please send questions or comments to CMB-S4 Science Council (sc@cmb-s4.org)."
contact = widgets.HTML(value = f"<font size='2px'>{con_text}")

sign_text = "Web-app development by Cooper Jacobus (cjacobus@berkeley.edu)."
sign =  widgets.HTML(value = f"<font size='2px'>{sign_text}")

output = widgets.Output()

def on_value_change(change):
    with output:
        if change['new'] == "Clusters: Chile Counts":
            z_bar.layout = top_bar_visible
        else:
            z_bar.layout = layout_none
            
            
        if change['new'] == 'Inflation: σ(r=0)':
            sidebar1.layout = layout_none
            sidebar2.layout = layout_block
        else:
            sidebar1.layout = layout_block
            sidebar2.layout = layout_none

w.children[13].observe(on_value_change, names='value')

def on_x_change(change):
    with output:
        
        if change['new'] == 'Detector Years':
            sidebar1.layout = layout_none
            sidebar2.layout = layout_none
            det_yrs = True
        else:
            sidebar1.layout = layout_block
            sidebar2.layout = layout_block
            
#w.children[-4].observe(on_x_change, names='value')

def bye_bye_button(change):
    w.children[-2].layout = layout_none
    
w.children[-2].observe(bye_bye_button)

def save_figure(change):
    make_plot.fig.savefig('Custom_Timeline.jpg')
    
w.children[-3].observe(save_figure)


display(VBox([title, subtitle, s1, window, s1, ack, contact, sign]))